# Problem Statement: Data Wrangling on Real Estate Market
● Import the "RealEstate_Prices.csv" dataset. 

● Handle missing values in the dataset, deciding on an appropriate strategy 

● Filter and subset the data based on specific criteria, such as a particular time period, property type, or location.

● Handle categorical variables by encoding them appropriately (e.g., one-hot encoding or label encoding) for further analysis.

● Aggregate the data to calculate summary statistics or derived metrics such as average sale prices by neighbourhood or property type.

● Identify and handle outliers or extreme values in the data that may affect the analysis or modelling process.

# Import the "RealEstate_Prices.csv" dataset. 

In [75]:
import pandas as pd

estate = pd.read_csv('../../Datasets/RealEstate_Prices.csv')
estate.head()

,property id,property type,neighborhood,bedrooms,bathrooms,area sq ft,year built,sale price,sale date
0,1001,Apartment,Downtown,3.0,2,1154.0,2001,341218.0,29-09-2020
1,1002,House,Suburban,4.0,3,2553.0,2014,490564.0,03-04-2021
2,1003,Condo,Urban,2.0,1,821.0,2008,NaN,15-04-2020
3,1004,House,Suburban,5.0,4,3018.0,2004,560852.0,25-08-2020
4,1005,Apartment,Downtown,2.0,1,976.0,2013,297985.0,21-05-2022


In [76]:
estate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property id    50 non-null     int64  
 1   property type  50 non-null     object 
 2   neighborhood   50 non-null     object 
 3   bedrooms       45 non-null     float64
 4   bathrooms      50 non-null     int64  
 5   area sq ft     48 non-null     float64
 6   year built     50 non-null     int64  
 7   sale price     45 non-null     float64
 8   sale date      50 non-null     object 
dtypes: float64(3), int64(3), object(3)
memory usage: 3.6+ KB


# Handle missing values in the dataset, deciding on an appropriate strategy 

In [77]:
estate.isna().sum()

property id      0
property type    0
neighborhood     0
bedrooms         5
bathrooms        0
area sq ft       2
year built       0
sale price       5
sale date        0
dtype: int64

In [79]:
num_cols = []
categorical_cols = []
for idx,val in estate.isna().sum().items():
    if val>0:
        if len(estate[idx].value_counts()) > 10:
            num_cols.append(idx)
        else:
            categorical_cols.append(idx)

In [80]:
num_cols

['area sq ft', 'sale price']

In [81]:
categorical_cols

['bedrooms']

In [82]:
for col in num_cols:
    mean = estate[col].mean()
    estate[col] = estate[col].fillna(mean)

for col in categorical_cols:
    mode = estate[col].mode()[0]
    estate[col] = estate[col].fillna(mode)

# Filter and subset the data based on specific criteria, such as a particular time period, property type, or location.

In [83]:
estate['sale date'] = pd.to_datetime(estate['sale date'],dayfirst=True)

In [84]:
estate[(estate['sale date'] > "01-01-2022") & (estate['property type']=='Apartment')]

,property id,property type,neighborhood,bedrooms,bathrooms,area sq ft,year built,sale price,sale date
4,1005,Apartment,Downtown,2.0,1,976.0,2013,297985.0,2022-05-21
7,1008,Apartment,Suburban,2.0,1,695.0,2022,193182.0,2022-02-03
14,1015,Apartment,Downtown,2.0,1,989.0,2018,299637.0,2022-11-16
24,1025,Apartment,Downtown,2.0,1,963.0,2012,317188.0,2023-04-09
34,1035,Apartment,Downtown,2.0,1,924.0,2018,349800.0,2022-12-30
44,1045,Apartment,Downtown,2.0,1,992.0,2012,304562.0,2022-04-13


# Handle categorical variables by encoding them appropriately (e.g., one-hot encoding or label encoding) for further analysis.

In [85]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
estate['property type'] = enc.fit_transform(estate['property type'])
estate['neighborhood'] = enc.fit_transform(estate['neighborhood'])

In [86]:
estate.head()

,property id,property type,neighborhood,bedrooms,bathrooms,area sq ft,year built,sale price,sale date
0,1001,0,0,3.0,2,1154.0,2001,341218.000000,2020-09-29
1,1002,2,2,4.0,3,2553.0,2014,490564.000000,2021-04-03
2,1003,1,3,2.0,1,821.0,2008,488715.466667,2020-04-15
3,1004,2,2,5.0,4,3018.0,2004,560852.000000,2020-08-25
4,1005,0,0,2.0,1,976.0,2013,297985.000000,2022-05-21


# Aggregate the data to calculate summary statistics or derived metrics such as average sale prices by neighbourhood or property type.

In [87]:
estate.groupby('neighborhood').agg({'sale price':'mean'})

,sale price
neighborhood,
0,317621.200000
1,568279.800000
2,418178.050000
3,846649.533333


In [88]:
enc.classes_

array(['Downtown', 'Rural', 'Suburban', 'Urban'], dtype=object)

# Identify and handle outliers or extreme values in the data that may affect the analysis or modelling process.

In [89]:
threashold = 3 * estate['sale price'].std()

In [90]:
outliers = estate[(estate['sale price'] > threashold) | (estate['sale price'] < -threashold)]
outliers

,property id,property type,neighborhood,bedrooms,bathrooms,area sq ft,year built,sale price,sale date
5,1006,1,3,3.0,2,1399.0,2020,1117439.0,2020-05-09
15,1016,1,3,3.0,2,1365.0,2021,1124017.0,2020-08-29
25,1026,1,3,3.0,2,1452.0,2013,1268442.0,2021-03-08
35,1036,1,3,3.0,2,1390.0,2019,1274015.0,2020-08-02
45,1046,1,3,3.0,2,20003.0,2020,1239005.0,2021-11-07


In [91]:
estate = estate[(estate['sale price'] < threashold) & (estate['sale price'] > -threashold)]
estate.shape

(45, 9)